Improvements:
* [X] Reverse input.
  * Method 1: Added in tkzer
  * Method 2: Added in `seq2seq` model's `forward` method
* [X] Train on smaller dataset (currently using just 32k/2M pairs) (lower epoch instead). New trainset with 64k/160k-pairs and 2k5 most freq words from original dataset
* [ ] Change init model to normal distribution from $-0.8$ to $0.8$

In [ ]:
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/4%20-%20Packed%20Padded%20Sequences%2C%20Masking%2C%20Inference%20and%20BLEU.ipynb
# based on https://gmihaila.github.io/tutorial_notebooks/pytorchtext_bucketiterator/#dataset-class

# Setup

In [1]:
!pip3 install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 881.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.8.2+cu111 which is incompatible.


In [2]:
import torch
print(torch.__version__)

1.8.2+cu111


In [3]:
!pip install torchtext==0.9 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.9.2+cu111 requires torch==1.8.2, but you have torch 1.8.0 which is incompatible.
torchaudio 0.8.2 requires torch==1.8.2, but you have torch 1.8.0 which is incompatible.


In [4]:
!python -m spacy download en_core_web_sm -q
!python -m spacy download de_core_news_sm -q
!python -m spacy download fr_core_news_sm -q

2023-02-27 18:25:19.596357: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 18:25:24.117122: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 18:25:24.117532: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 18:25:24.117571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy.data import Dataset, Example

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time
import pickle
from tqdm import tqdm

# Work

## Tutorial

In [ ]:
# SEED = 1234
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

In [ ]:
spacy_de = spacy.load('de_core_news_sm')
# spacy_fr = spacy.load('fr_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]
# def tokenize_fr(text):
#     return [tok.text for tok in spacy_fr.tokenizer(text)]
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            include_lengths = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 422kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 113kB/s] 


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 108kB/s] 


In [ ]:
len(train_data)

29000

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

In [ ]:
for i, batch in enumerate(train_iterator):
  break

In [ ]:
src, src_len = batch.src
src

tensor([[   2,    2,    2,    2],
        [   5,   18,    5,    8],
        [  25,  103,  496,   36],
        [ 124,  274,   10,   22],
        [ 646,   25,    5,   54],
        [   7,   11,  558,   69],
        [ 172,  827,  738,   12],
        [5072,   53,   11,   14],
        [  42,   12,   18,  247],
        [  33,    6,    0,   12],
        [  51,   50,   57,   10],
        [1356, 1816,  675,  284],
        [   4,    4,    4,    4],
        [   3,    3,    3,    3]], device='cuda:0')

In [ ]:
packed = nn.utils.rnn.pack_padded_sequence(src, src_len.to('cpu'))
packed

PackedSequence(data=tensor([   2,    2,    2,    2,    5,   18,    5,    5,   13,   30,  164,  618,
          10,   10,   31,    7,    5,    8,    7,  661,   70,   16,    6,  998,
          26,  141, 2329,   68,  983,  133,   10,   19,    7,    7,   38,    0,
           6,    6,   11,  100,  268, 1019,  111,  877,    4,    4,    4,    4,
           3,    3,    3,    3], device='cuda:0'), batch_sizes=tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]), sorted_indices=None, unsorted_indices=None)

In [ ]:
outputs, lens = nn.utils.rnn.pad_packed_sequence(packed)
outputs, lens

(tensor([[   2,    2,    2,    2],
         [   5,   18,    5,    5],
         [  13,   30,  164,  618],
         [  10,   10,   31,    7],
         [   5,    8,    7,  661],
         [  70,   16,    6,  998],
         [  26,  141, 2329,   68],
         [ 983,  133,   10,   19],
         [   7,    7,   38,    0],
         [   6,    6,   11,  100],
         [ 268, 1019,  111,  877],
         [   4,    4,    4,    4],
         [   3,    3,    3,    3]], device='cuda:0'), tensor([13, 13, 13, 13]))

### Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
                
        #need to explicitly put lengths on cpu!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,518,405 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Train func

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src, src_len = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, src_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len, trg, 0) #turn off teacher forcing
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Train

In [ ]:
for batch in train_iterator:
  break
batch.src[0].shape

torch.Size([11, 128])

In [ ]:
TRG.vocab.stoi["<sos>"], TRG.vocab.stoi["<eos>"], TRG.vocab.stoi["<pad>"], TRG.vocab.itos[0]

(2, 3, 1, '<unk>')

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = 0
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 51s
	Train Loss: 2.358 | Train PPL:  10.565
	 Val. Loss: 0.000 |  Val. PPL:   1.000


KeyboardInterrupt: ignored

In [ ]:
# Epoch: 01 | Time: 0m 57s
# 	Train Loss: 4.716 | Train PPL: 111.752
# 	 Val. Loss: 4.335 |  Val. PPL:  76.338
# Epoch: 02 | Time: 0m 57s
# 	Train Loss: 3.406 | Train PPL:  30.133
# 	 Val. Loss: 3.571 |  Val. PPL:  35.548
# Epoch: 03 | Time: 0m 56s
# 	Train Loss: 2.705 | Train PPL:  14.957
# 	 Val. Loss: 3.321 |  Val. PPL:  27.686
# Epoch: 04 | Time: 0m 57s
# 	Train Loss: 2.288 | Train PPL:   9.858
# 	 Val. Loss: 3.225 |  Val. PPL:  25.160
# Epoch: 05 | Time: 0m 57s
# 	Train Loss: 2.003 | Train PPL:   7.414
# 	 Val. Loss: 3.141 |  Val. PPL:  23.123
# Epoch: 06 | Time: 0m 57s
# 	Train Loss: 1.755 | Train PPL:   5.786
# 	 Val. Loss: 3.186 |  Val. PPL:  24.185

### Inference

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
  model.eval()
  # create tensor from sent
  if isinstance(sentence, str):
    nlp = spacy.load('fr_core_news_sm')
    tokens = [token.text.lower() for token in nlp(sentence)]
  else:
    tokens = [token.lower() for token in sentence]
  tokens = [src_field.init_token] + tokens + [src_field.eos_token]       
  src_indexes = [src_field.vocab.stoi[token] for token in tokens]    
  src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)  # [seq_len, N] w/ N=1 for batch
  src_len = torch.LongTensor([len(src_indexes)])

  # This part can be replaced w/ model directly
  with torch.no_grad():
    encoder_outputs, hidden = model.encoder(src_tensor, src_len)

  mask = model.create_mask(src_tensor)
  # This is == trg
  trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
  attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
  for i in range(max_len):
    trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
    with torch.no_grad():
      output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)  # output = [batch size, output dim]

    attentions[i] = attention
    pred_token = output.argmax(1).item()
    trg_indexes.append(pred_token)
    if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
      break
  
  trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
  return trg_tokens[1:], attentions[:len(trg_tokens)-1]

In [ ]:
example_idx = 12
src = vars(valid_dt.examples[example_idx])['src']
trg = vars(valid_dt.examples[example_idx])['trg']
print(f'src = {src}')
print(f'trg = {trg}')

In [ ]:
translation, attention = translate_sentence(src, SRC_FIELD, TRG_FIELD, model_infer, device)
print(f'predicted trg = {translation}')

### BLEU

In [ ]:
from torchtext.data.metrics import bleu_score
def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
  trgs = []
  pred_trgs = []
  for datum in tqdm(data):
    src = vars(datum)['src']
    trg = vars(datum)['trg']
    pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
    #cut off <eos> token
    pred_trg = pred_trg[:-1]
    pred_trgs.append(pred_trg)
    trgs.append([trg])
  return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu(test_dt, SRC_FIELD, TRG_FIELD, model_infer, device)
print(f'BLEU score = {bleu_score*100:.2f}')

# My Section

## Setup

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
spacy_de = spacy.load('de_core_news_sm')
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

spacy_fr = spacy.load('fr_core_news_sm')
def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]

spacy_en = spacy.load('en_core_web_sm')
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [8]:
SRC_FIELD = Field(tokenize = tokenize_en, 
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True,
            include_lengths = True
            )
PIV_FIELD = Field(tokenize = tokenize_de,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True,
            include_lengths = True
            )
TRG_FIELD = Field(tokenize = tokenize_fr,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True
            )

## Data

In [9]:
# with open('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/enfr_160kpairs_2k5-freq-words.pkl', 'rb') as f:
#   data = pickle.load(f)
#   random.shuffle(data)
# data[80], len(data)

In [10]:
with open('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/endefr_75kpairs_2k5-freq-words.pkl', 'rb') as f:
  data = pickle.load(f)
  random.shuffle(data)
data[8], len(data)

({'en': 'in writing.',
  'de': 'schriftlich.',
  'fr': "J'ai voté en faveur de cette résolution."},
 75002)

In [11]:
train_len = 64000
valid_len = 3200
test_len = 6400

train_pt = train_len
valid_pt = train_pt + valid_len
test_pt = valid_pt + test_len

In [12]:
# For 2 langs
# data_set = [[pair['en'].lower(), pair['fr'].lower()] for pair in data]
# FIELDS = [('src', EN_SRC), ('trg', FR_TRG)]
# train_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[: train_pt]))
# valid_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[train_pt : valid_pt]))
# test_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[valid_pt : test_pt]))

# For 3 langs
data_set = [[pair['en'], pair['de'], pair['fr']] for pair in data]
FIELDS = [('src', SRC_FIELD), ('piv', PIV_FIELD), ('trg', TRG_FIELD)]
train_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[: train_pt]))
valid_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[train_pt : valid_pt]))
test_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[valid_pt : test_pt]))

In [13]:
train_dt = Dataset(train_examples, fields=FIELDS)
valid_dt = Dataset(valid_examples, fields=FIELDS)
test_dt = Dataset(test_examples, fields=FIELDS)

In [14]:
SRC_FIELD.build_vocab(train_dt, min_freq = 1) # choose 1 since data is already filter w/ most freq words
PIV_FIELD.build_vocab(train_dt, min_freq = 1)
TRG_FIELD.build_vocab(train_dt, min_freq = 1)
len(SRC_FIELD.vocab), len(PIV_FIELD.vocab), len(TRG_FIELD.vocab)  # should be 2k5

(2272, 2474, 2388)

In [15]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dt, valid_dt, test_dt),
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

In [16]:
# for i, batch in enumerate(train_iterator):
#   print(batch.src[0].shape, batch.src[1], batch.piv[0].shape, batch.piv[1], batch.trg.shape)
#   if i==2: break
# batch.fields

In [17]:
# batch.src

In [18]:
# batch.trg

In [19]:
# src_sent, piv_sent, trg_sent = [], [], []
# for i in batch.src[0][: , 0]:
#   src_sent.append(SRC_FIELD.vocab.itos[i])
# for i in batch.piv[0][: , 0]:
#   piv_sent.append(PIV_FIELD.vocab.itos[i])
# for i in batch.trg[:, 0]:
#   trg_sent.append(TRG_FIELD.vocab.itos[i])
# print(' '.join(src_sent))
# print(' '.join(piv_sent))
# print(' '.join(trg_sent))

In [20]:
# for i in range(5):
#   print(SRC_FIELD.vocab.itos[i], PIV_FIELD.vocab.itos[i], TRG_FIELD.vocab.itos[i])

In [21]:
# batch.src[0][:, 0]

## Model

### Encoder

In [51]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
    self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, src, src_len):
    #src = [src len, batch size]
    #src_len = [batch size]
    embedded = self.dropout(self.embedding(src))  #embedded = [src len, batch size, emb dim]

    #need to explicitly put lengths on cpu!
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))

    #  when the input is a pad token are all zeros
    packed_outputs, hidden = self.rnn(packed_embedded)
    #packed_outputs is a packed sequence containing all hidden states
    #hidden is now from the final non-padded element in the batch

    outputs, len_list = nn.utils.rnn.pad_packed_sequence(packed_outputs) #outputs is now a non-packed sequence, all hidden states obtained
    #  when the input is a pad token are all zeros
    
    #outputs = [src len, batch size, hid dim * num directions]
    #hidden = [n layers * num directions, batch size, hid dim]
    
    #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
    #outputs are always from the last layer
    
    #hidden [-2, :, : ] is the last of the forwards RNN 
    #hidden [-1, :, : ] is the last of the backwards RNN
    
    #initial decoder hidden is final hidden state of the forwards and backwards 
    #  encoder RNNs fed through a linear layer
    hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

    #outputs = [src len, batch size, enc hid dim * 2]
    #hidden = [batch size, dec hid dim]
    return outputs, hidden

### Attn

In [52]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()
    self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
    self.v = nn.Linear(dec_hid_dim, 1, bias = False)
      
  def forward(self, hidden, encoder_outputs, mask):
    #hidden = [batch size, dec hid dim]
    #encoder_outputs = [src len, batch size, enc hid dim * 2]
    batch_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]

    #repeat decoder hidden state src_len times
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  #hidden = [batch size, src len, dec hid dim]
    encoder_outputs = encoder_outputs.permute(1, 0, 2)  #encoder_outputs = [batch size, src len, enc hid dim * 2]
    energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) #energy = [batch size, src len, dec hid dim]

    attention = self.v(energy).squeeze(2) #attention = [batch size, src len]
    attention = attention.masked_fill(mask == 0, -1e10)
    return F.softmax(attention, dim = 1)

### Decoder

In [53]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
    super().__init__()
    self.output_dim = output_dim
    self.attention = attention
    self.embedding = nn.Embedding(output_dim, emb_dim)
    self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
    self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
    self.dropout = nn.Dropout(dropout)
      
  def forward(self, input, hidden, encoder_outputs, mask):
    #input = [batch size]
    #hidden = [batch size, dec hid dim]
    #encoder_outputs = [src len, batch size, enc hid dim * 2]
    #mask = [batch size, src len]
    input = input.unsqueeze(0)  #input = [1, batch size]
    embedded = self.dropout(self.embedding(input))  #embedded = [1, batch size, emb dim]

    a = self.attention(hidden, encoder_outputs, mask) #a = [batch size, src len]
    a = a.unsqueeze(1)  #a = [batch size, 1, src len]

    encoder_outputs = encoder_outputs.permute(1, 0, 2)  #encoder_outputs = [batch size, src len, enc hid dim * 2]

    weighted = torch.bmm(a, encoder_outputs)  #weighted = [batch size, 1, enc hid dim * 2]
    weighted = weighted.permute(1, 0, 2)  #weighted = [1, batch size, enc hid dim * 2]

    rnn_input = torch.cat((embedded, weighted), dim = 2)  #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

    output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
    #output = [seq len, batch size, dec hid dim * n directions]
    #hidden = [n layers * n directions, batch size, dec hid dim]
    
    #seq len, n layers and n directions will always be 1 in this decoder, therefore:
    #output = [1, batch size, dec hid dim]
    #hidden = [1, batch size, dec hid dim]
    #this also means that output == hidden
    assert (output == hidden).all()
    
    embedded = embedded.squeeze(0)
    output = output.squeeze(0)
    weighted = weighted.squeeze(0)
    
    prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))  #prediction = [batch size, output dim]
    return prediction, hidden.squeeze(0), a.squeeze(1)

### Seq2Seq

In [54]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, src_pad_idx, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_pad_idx = src_pad_idx
    self.device = device
      
  def create_mask(self, src):
    mask = (src != self.src_pad_idx).permute(1, 0)
    return mask
      
  def forward(self, src, src_len, trg, criterion=None, teacher_forcing_ratio = 0.5):
    #src = [src len, batch size]
    #src_len = [batch size]
    #trg = [trg len, batch size]
    #teacher_forcing_ratio is probability to use teacher forcing. E.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
    
    batch_size = src.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim
    
    #tensor to store decoder outputs
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
    
    #encoder_outputs is all hidden states of the input sequence, back and forwards
    #hidden is the final forward and backward hidden states, passed through a linear layer
    encoder_outputs, hidden = self.encoder(src, src_len)

    #first input to the decoder is the <sos> tokens
    input = trg[0,:]
    
    mask = self.create_mask(src)  #mask = [batch size, src len]
            
    for t in range(1, trg_len):
      #insert input token embedding, previous hidden state, all encoder hidden states and mask
      #receive output tensor (predictions) and new hidden state
      output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
      
      #place predictions in a tensor holding predictions for each token
      outputs[t] = output
      
      #if teacher forcing, use actual next token as next input. Else, use predicted token
      input = trg[t] if random.random() < teacher_forcing_ratio else output.argmax(1)
    
    if criterion != None:
      return self.compute_loss(outputs, trg, criterion), outputs
    return outputs
  
  def compute_loss(self, output, trg, criterion):
    #output = (trg_len, batch_size, trg_vocab_size)
    #trg = [trg len, batch size]
    output_dim = output.shape[-1]
    output = output[1:].view(-1, output_dim)  #output = [(trg len - 1) * batch size, output dim]
    trg = trg[1:].view(-1)  #trg = [(trg len - 1) * batch size]
    return criterion(output, trg)

### Pivot model

In [55]:
class PivotSeq2Seq(nn.Module):
  def __init__(self, model1, model2, src_field, piv_field, trg_field, device, lamda=0.75):
    super().__init__()
    self.model1 = model1
    self.model2 = model2
    self.src_field = src_field
    self.piv_field = piv_field
    self.trg_field = trg_field
    self.device = device
    self.lamda = lamda
      
  def forward(self, src, src_len, piv, piv_len, trg, criterion=None, teacher_forcing_ratio=0.5):
    #src = [src len, batch size]
    #src_len = [batch size]
    #piv = [piv len, batch size]
    #trg = [trg len, batch size]
    if criterion != None:
      criterion1, criterion2 = criterion
      loss1, output1 = self.model1(src, src_len, piv, criterion1, teacher_forcing_ratio) # output1 = [piv len, batch size, output dim]

      if random.random() < teacher_forcing_ratio:
        piv, piv_len, trg = self.process_output(output1, trg) # piv = [piv len, batch_size] & piv_len = [batch_size]
      else:
        piv, piv_len, trg = self.sort_by_piv(piv, piv_len, trg)

      loss2, output2 = self.model2(piv, piv_len, trg, criterion2, teacher_forcing_ratio) # output2 = [trg len, batch size, output dim]
      return self.compute_loss(loss1, loss2), output2
    
    output1 = self.model1(src, src_len, piv, teacher_forcing_ratio)
    if random.random() < teacher_forcing_ratio:
      piv, piv_len, trg = self.process_output(output1, trg)
    else:
      piv, piv_len, trg = self.sort_by_piv(piv, piv_len, trg)
    output2 = self.model2(piv, piv_len, trg, teacher_forcing_ratio)

    return output2
  
  def compute_loss(self, loss1, loss2):
    return loss1 + loss2 + self.lamda*self.compute_embed_loss()
  
  def compute_embed_loss(self):
    # decoder.embedding.weight = [piv_vocab_dim, emb_dim]
    # F.pairwise_distance = [piv_vocab_dim] --> distance for each vector along emb_dim
    return torch.sum(F.pairwise_distance(self.model1.decoder.embedding.weight, self.model2.encoder.embedding.weight, p=2))
  
  def sort_by_piv(self, piv, piv_len, trg):
    piv_len, sorted_ids = piv_len.sort(descending=True)
    return piv[:, sorted_ids], piv_len, trg[:, sorted_ids]

  def process_output(self, output, trg):
    # output = [trg len, batch size, output dim]
    # trg = [trg len, batch size]
    # Process output1 to be input for model2
    seq_len, N, _ = output.shape
    tmp_out = output.argmax(2)  # tmp_out = [seq_len, batch_size]
    # re-create piv as src for model2
    piv = torch.zeros_like(tmp_out).type(torch.long).to(output.device)
    piv[0, :] = torch.full_like(piv[0, :], self.piv_field.vocab.stoi[self.piv_field.init_token])  # fill all first idx with sos_token
    
    for i in range(1, tmp_out.shape[0]):  # for each i in seq_len
      # if tmp_out's prev is eos_token, replace w/ pad_token, else current value
      eos_mask = tmp_out[i-1, :] == self.piv_field.vocab.stoi[self.piv_field.eos_token]
      piv[i, :] = torch.where(eos_mask, self.piv_field.vocab.stoi[self.piv_field.pad_token], tmp_out[i, :])
      # if piv's prev is pad_token, replace w/ pad_token, else current value
      pad_mask = (piv[i-1, :] == self.piv_field.vocab.stoi[self.piv_field.pad_token])
      piv[i, :] = torch.where(pad_mask, self.piv_field.vocab.stoi[self.piv_field.pad_token], piv[i, :])
    
    # Trim down extra pad tokens
    tensor_list = [piv[i] for i in range(seq_len) if not all(piv[i] == self.piv_field.vocab.stoi[self.piv_field.pad_token])]  # tensor_list = [new_seq_len, batch_size]
    piv = torch.stack([x for x in tensor_list], dim=0).type(torch.long).to(output.device)
    
    # get seq_id + eos_tok id of each sequence
    piv_ids, eos_ids = (piv.permute(1, 0) == self.piv_field.vocab.stoi[self.piv_field.eos_token]).nonzero(as_tuple=True)  # piv_len = [N]
    piv_len = torch.full_like(piv[0], piv.shape[0]).type(torch.long)  # fill w/ seq_len in case no <eos> found
    piv_len[piv_ids] = eos_ids + 1 # include eos tok in len # still works even when piv_ids, piv_len = tensor([], dtype=...)
      
    assert not all(piv[-1] == self.piv_field.vocab.stoi[self.piv_field.pad_token]), 'Not completely trim down tensor'

    # piv = [seq_len, batch_size]
    piv, piv_len, trg = self.sort_by_piv(piv, piv_len, trg)
    return piv, piv_len, trg

## Train func

In [42]:
def train(model, iterator, optimizer, criterion, clip, isPivot=False, force=0.5):
  model.train()
  epoch_loss = 0.0
  
  for batch in tqdm(iterator):
    src, src_len = batch.src
    trg = batch.trg # trg = [trg len, batch size]

    optimizer.zero_grad()
    
    if not isPivot:
      loss, output = model(src, src_len, trg, criterion)
    else:
      assert len(criterion) == 2, 'Need 2 criterions in Tuple for each model'
      piv, piv_len = batch.piv # piv = [piv len, batch size]
      loss, output = model(src, src_len, piv, piv_len, trg, criterion, force) # criterion = (criterion1, criterion2)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(iterator)

In [43]:
def evaluate(model, iterator, criterion, isPivot=False, force=0):
  model.eval()
  epoch_loss = 0.0
  
  with torch.no_grad():
    for batch in tqdm(iterator):
      src, src_len = batch.src
      trg = batch.trg # trg = [trg len, batch size]

      if not isPivot:
        loss, output = model(src, src_len, trg, criterion, force) # turn off teacher forcing
      else:
        assert len(criterion) == 2, 'Need 2 criterions in Tuple for each model'
        piv, piv_len = batch.piv # piv = [piv len, batch size]
        loss, output = model(src, src_len, piv, piv_len, trg, criterion, force) # criterion = (criterion1, criterion2)
      
      epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [44]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [45]:
def update_trainlog(data: list, filename: str='/content/gdrive/MyDrive/Colab Notebooks/eaai24/training_log.txt'):
  ''' Update training log w/ new losses
  Args:
      data (List): a list of infor for many epochs as tuple, each tuple has model_name, loss, etc.
      filename (String): path + file_name
  Return:
      None: new data is appended into train-log
  '''
  with open(filename, 'a') as f: # save
    for epoch in data:
      f.write(','.join(epoch))
      f.write("\n")
  print('update_trainlog SUCCESS')
  return []

## Train

In [ ]:
# # For 2 langs
# INPUT_DIM = len(SRC_FIELD.vocab)
# OUTPUT_DIM = len(TRG_FIELD.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5
# LR = 0.001
# SRC_PAD_IDX = SRC_FIELD.vocab.stoi[SRC_FIELD.pad_token]

# attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
# enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
# dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

# model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [56]:
# For 3 langs
INPUT_DIM = len(SRC_FIELD.vocab)
PIV_DIM = len(PIV_FIELD.vocab)
OUTPUT_DIM = len(TRG_FIELD.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LR = 0.001

SRC_PAD_IDX = SRC_FIELD.vocab.stoi[SRC_FIELD.pad_token]
PIV_PAD_IDX = PIV_FIELD.vocab.stoi[PIV_FIELD.pad_token]

attn1 = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc1 = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec1 = Decoder(PIV_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn1)
model1 = Seq2Seq(enc1, dec1, SRC_PAD_IDX, device).to(device)

attn2 = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc2 = Encoder(PIV_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec2 = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn2)
model2 = Seq2Seq(enc2, dec2, PIV_PAD_IDX, device).to(device)

model = PivotSeq2Seq(model1, model2, SRC_FIELD, PIV_FIELD, TRG_FIELD, device).to(device)

In [57]:
def init_weights(m):
  for name, param in m.named_parameters():
    if 'weight' in name:
      nn.init.normal_(param.data, mean=0, std=0.01)
    else:
      nn.init.constant_(param.data, 0)
            
model.apply(init_weights);

In [58]:
optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html#torch.optim.lr_scheduler.StepLR
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [59]:
criterion1 = nn.CrossEntropyLoss(ignore_index = PIV_FIELD.vocab.stoi[PIV_FIELD.pad_token])
criterion2 = nn.CrossEntropyLoss(ignore_index = TRG_FIELD.vocab.stoi[TRG_FIELD.pad_token])
criterions = (criterion1, criterion2)

In [60]:
N_EPOCHS = 5
CLIP = 1
best_valid_loss = float('inf')
best_train_loss = float('inf')
model_name = 'piv_endefr_74kset.pt'
train_log = []

In [61]:
for epoch in range(N_EPOCHS):
  start_time = time.time()
  # For 2 langs
  # train_loss = train(model, train_iterator, optimizer, criterions, CLIP)
  # valid_loss = evaluate(model, valid_iterator, criterions)

  # For 3 langs
  train_loss = train(model, train_iterator, optimizer, criterions, clip=CLIP, isPivot=True, force=0.5)
  valid_loss = evaluate(model, valid_iterator, criterions, isPivot=True, force=0)
  scheduler.step()  # following this: https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html#torch.optim.lr_scheduler.StepLR

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  
  epoch_info = [model_name, LR, BATCH_SIZE, ENC_HID_DIM, 'no-num_layers', ENC_DROPOUT, DEC_DROPOUT, epoch, N_EPOCHS, train_loss, valid_loss]
  train_log.append([str(ele) for ele in epoch_info])

  if train_loss < best_train_loss or valid_loss < best_valid_loss:
    best_train_loss = train_loss
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), f'/content/gdrive/MyDrive/Colab Notebooks/eaai24/{model_name}')
    print('SAVED MODEL')
    train_log = update_trainlog(train_log)
  
  print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

100%|██████████| 50/50 [00:02<00:00, 17.17it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 01 | Time: 2m 42s
	Train Loss: 14.412 | Train PPL: 1816013.888
	 Val. Loss: 9.519 |  Val. PPL: 13618.213


100%|██████████| 50/50 [00:03<00:00, 14.83it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 02 | Time: 2m 46s
	Train Loss: 8.149 | Train PPL: 3459.407
	 Val. Loss: 8.562 |  Val. PPL: 5229.127


100%|██████████| 50/50 [00:03<00:00, 16.04it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 03 | Time: 2m 44s
	Train Loss: 7.402 | Train PPL: 1639.707
	 Val. Loss: 8.140 |  Val. PPL: 3430.549


100%|██████████| 50/50 [00:03<00:00, 15.92it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 04 | Time: 2m 46s
	Train Loss: 6.872 | Train PPL: 964.566
	 Val. Loss: 8.000 |  Val. PPL: 2981.026


100%|██████████| 50/50 [00:02<00:00, 18.73it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 05 | Time: 2m 45s
	Train Loss: 6.524 | Train PPL: 681.343
	 Val. Loss: 7.800 |  Val. PPL: 2440.980


In [62]:
for epoch in range(N_EPOCHS, N_EPOCHS+5):
  start_time = time.time()
  # For 2 langs
  # train_loss = train(model, train_iterator, optimizer, criterions, CLIP)
  # valid_loss = evaluate(model, valid_iterator, criterions)

  # For 3 langs
  train_loss = train(model, train_iterator, optimizer, criterions, clip=CLIP, isPivot=True, force=0)
  valid_loss = evaluate(model, valid_iterator, criterions, isPivot=True, force=0)

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  
  epoch_info = [model_name, LR, BATCH_SIZE, ENC_HID_DIM, 'no-num_layers', ENC_DROPOUT, DEC_DROPOUT, epoch, N_EPOCHS, train_loss, valid_loss]
  train_log.append([str(ele) for ele in epoch_info])

  if train_loss < best_train_loss or valid_loss < best_valid_loss:
    best_train_loss = train_loss
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), f'/content/gdrive/MyDrive/Colab Notebooks/eaai24/{model_name}')
    print('SAVED MODEL')
    train_log = update_trainlog(train_log)
  
  print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

100%|██████████| 50/50 [00:03<00:00, 15.65it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 06 | Time: 2m 43s
	Train Loss: 7.038 | Train PPL: 1139.269
	 Val. Loss: 7.228 |  Val. PPL: 1377.404


100%|██████████| 50/50 [00:03<00:00, 15.56it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 07 | Time: 2m 43s
	Train Loss: 6.681 | Train PPL: 796.978
	 Val. Loss: 7.145 |  Val. PPL: 1267.301


100%|██████████| 50/50 [00:03<00:00, 16.60it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 08 | Time: 2m 43s
	Train Loss: 6.451 | Train PPL: 633.175
	 Val. Loss: 7.102 |  Val. PPL: 1214.458


100%|██████████| 50/50 [00:03<00:00, 15.90it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 09 | Time: 2m 42s
	Train Loss: 6.244 | Train PPL: 514.798
	 Val. Loss: 7.084 |  Val. PPL: 1192.685


100%|██████████| 50/50 [00:03<00:00, 15.69it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 10 | Time: 2m 42s
	Train Loss: 6.062 | Train PPL: 429.278
	 Val. Loss: 7.124 |  Val. PPL: 1241.634


## Eval

In [ ]:
# FIELDS = [('src', EN_SRC), ('trg', FR_TRG)]
# start_idx = train_len + valid_len
# end_idx = start_idx + 6400
# test_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[start_idx : end_idx]))
# test_dt = Dataset(test_examples, fields=FIELDS)
# test_iterator1 = BucketIterator(
#     test_dt,
#      batch_size = BATCH_SIZE,
#      sort_within_batch = True,
#      sort_key = lambda x : len(x.src),
#      device = device)

In [ ]:
for batch in test_iterator:
  break
batch.fields

dict_keys(['src', 'trg'])

In [ ]:
INPUT_DIM = len(SRC_FIELD.vocab)
OUTPUT_DIM = len(TRG_FIELD.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LR = 0.001
SRC_PAD_IDX = SRC_FIELD.vocab.stoi[SRC_FIELD.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model_infer = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [ ]:
# model_name = 'attn_en-fr_32k_160kset_invAfPad.pt'
model_infer.load_state_dict(torch.load(f'/content/gdrive/MyDrive/Colab Notebooks/eaai24/{model_name}'))

<All keys matched successfully>

In [ ]:
test_loss = evaluate(model_infer, test_iterator, criterion1)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

<All keys matched successfully>

## Inference

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
  model.eval()
  # create tensor from sent
  if isinstance(sentence, str):
    nlp = spacy.load('fr_core_news_sm')
    tokens = [token.text.lower() for token in nlp(sentence)]
  else:
    tokens = [token.lower() for token in sentence]
  tokens = [src_field.init_token] + tokens + [src_field.eos_token]       
  src_indexes = [src_field.vocab.stoi[token] for token in tokens]    
  src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)  # [seq_len, N] w/ N=1 for batch
  src_len = torch.LongTensor([len(src_indexes)])

  # This part can be replaced w/ model directly
  with torch.no_grad():
    encoder_outputs, hidden = model.encoder(src_tensor, src_len)

  mask = model.create_mask(src_tensor)
  # This is == trg
  trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
  attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
  for i in range(max_len):
    trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
    with torch.no_grad():
      output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)  # output = [batch size, output dim]

    attentions[i] = attention
    pred_token = output.argmax(1).item()
    trg_indexes.append(pred_token)
    if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
      break
  
  trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
  return trg_tokens[1:], attentions[:len(trg_tokens)-1]

In [ ]:
# To modify translate_sentence function ===> Using model directly
def forward(self, src, src_len, trg, criterion=None, teacher_forcing_ratio = 0):
      #src = [src len, 1]
      #src_len = [1]
      #trg = [trg len, 1]
      src = torch.flip(src, (0,))  # flip src_len [<sos>, ... , <eos>, <pad>, <pad>] ==> [<pad>, <pad>, <eos>, ... , <sos>]
      batch_size = src.shape[1]
      trg_len = trg.shape[0]
      trg_vocab_size = self.decoder.output_dim
      
      outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
      encoder_outputs, hidden = self.encoder(src, src_len)

      input = trg[0,:]
      mask = self.create_mask(src)  #mask = [batch size, src len]
              
      for t in range(1, trg_len):
          output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
          outputs[t] = output
          input = trg[t] if random.random() < teacher_forcing_ratio else output.argmax(1)      
      return outputs

In [ ]:
example_idx = 12
src = vars(valid_dt.examples[example_idx])['src']
trg = vars(valid_dt.examples[example_idx])['trg']
print(f'src = {src}')
print(f'trg = {trg}')

src = ['.', 'closed', 'is', 'debate', 'the']
trg = ['le', 'débat', 'est', 'clos', '.']


In [ ]:
translation, attention = translate_sentence(src, SRC_FIELD, TRG_FIELD, model_infer, device)
print(f'predicted trg = {translation}')

predicted trg = ['le', 'débat', 'est', 'clos', '.', '<eos>']


## BLEU

In [ ]:
from torchtext.data.metrics import bleu_score
def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
  trgs = []
  pred_trgs = []
  for datum in tqdm(data):
    src = vars(datum)['src']
    trg = vars(datum)['trg']
    pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
    #cut off <eos> token
    pred_trg = pred_trg[:-1]
    pred_trgs.append(pred_trg)
    trgs.append([trg])
  return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu(test_dt, SRC_FIELD, TRG_FIELD, model_infer, device)
print(f'BLEU score = {bleu_score*100:.2f}')

100%|██████████| 3200/3200 [00:48<00:00, 65.34it/s]


BLEU score = 32.15


* attn_en-fr_32k.pt: BLEU = 12.65
* attn_en-fr_32k_160kset_inverse.pt: BLEU = 32.15
* attn_en-fr_32k_160kset_invAfpad.pt: causes data error in pack_padded_sequence --> must not flip


# End